## Evaluate ESG scores for bonds portfolio

In [ ]:
# load imports
import pandas as pd
import json, math
import plotly.graph_objects as go

In [ ]:
# execute the helper functions defined for accessing RDP REST API calls 
%run RDPDefines.ipynb

### What is Symbology Mapping

Symbology Endpoint: [api.refinitiv.com/discovery/symbology/v1/lookup]()

parameters [route = FindESGStatementParent]()

<br/>
Examples:
<br/>
<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 44654483026        |         |
| Issuer     | SAUDI ELECTRICITY GLOBAL SUKUK COMPANY 3        |         |
| 1st Parent | 4295887339 (Saudi Electricity Company)        |    ✔     |
| Result     | 4295887339        |         |

<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 192814833479        |         |
| Issuer     | GOLDMAN SACHS FINANCE CORP INTERNATIONAL LTD        |         |
| 1st Parent | GS Global Markets Inc        |         |
| 2nd Parent | 4295911963 (Goldman Sachs Group Inc)        |    ✔   |
| Result     | 4295911963       |         |

<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 192846098875        |         |
| Issuer     | MORGAN STANLEY BANK NA        |         |
| 1st Parent | MORGAN STANLEY DOMESTIC HOLDINGS INC        |         |
| 2nd Parent | Morgan Stanley Capital Management LLC        |         |
| 3rd Parent | 4295904557 (Morgan Stanley)        |    ✔   |
| Result     | 4295904557        |         |

<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 46641173275        |         |
| Issuer     | PROPERTY AND BUILDING CORP LTD        |         |
| 1st Parent | DISCOUNT INVESTMENT CORP LTD        |         |
| 2nd Parent | DOLPHIN NETHERLANDS BV        |         |
| 3rd Parent | TYRUS SA        |         |
| 4th Parent | 5000620306 (RSA INVERSIONES Y REPRESENTACIONES SA)        |    ✔    |
| Result     | 5000620306        |         |


### Download the bulk JSON files

In [ ]:
# define the download function
def downloadJSONBulkFile(bucketName, fileAttributes, fileNameKeywords):
    # get a list of all the buckets
    hResp = getRequest('/file-store/v1/file-sets?bucket=' + bucketName + '&pageSize=100&attributes=' + fileAttributes)
    print(hResp)
    # loop through all the buckets
    for bucket in hResp['value']:
        bName = bucket['name']
        # does bucket contains all the matching keywords
        if all([x in bName for x in fileNameKeywords]):
            fileName = bucket['files'][0]
            print('Found bucket: ', bName, ', FileName: ', fileName)
            # stop any more searching
            break
    
    if not fileName:
        raise Exception('No matching bulk file found in bucket:'.format(bucketName))

    # download and uncompress the file object
    fileStr = downloadUncompressFile('/file-store/v1/files/' + fileName + '/stream')
    print('File downloaded and uncompressed, size: ', len(fileStr))
    return fileStr


#### Download and save the Bond ISIN - ESG Parent mapping

In [ ]:
# download the Bond-ESGParent symbology database
jsonlFile = downloadJSONBulkFile('bulk-symbology', 'ContentType:Symbology BondISINSusFinMapping', ['Bond', 'ISIN', 'Json', 'Init'])
# parse out the entries in the bulk file
mapping = []
for l in jsonlFile.splitlines():
    jObj = json.loads(l)
    if len(jObj['Identifiers']) > 0 and jObj['EsgCoverage']['EsgStatementParentOrganization']:
        coName = jObj['EsgCoverage']['EsgStatementParentOrganization']['PartyName']['Names'][0]['NormalizedName'] if jObj['EsgCoverage']['EsgStatementParentOrganization']['PartyName']['Names'] else ''
        mapping.append((jObj['Identifiers'][0]['IdentifierValue'], jObj['EsgCoverage']['EsgStatementParentOrganization']['ObjectId'], coName))
    
print('Loaded {} Bonds ISIN to ESG Parent PermID mappings'.format(len(mapping)))

In [ ]:
# load the dataset into a pandas dataframe
df1 = pd.DataFrame(mapping, columns=['Bond', 'ESGParent', 'ParentName'])
# save the database
df1.to_pickle('Bond_Parent_mapping.pkl')

#### Download and save the ESG Scores dataset

In [ ]:
# download the ESG Scores database
jsonlFile = downloadJSONBulkFile('bulk-ESG', 'ContentType:ESG Scores', ['Scores-Full', 'Init', 'Jsonl'])

In [ ]:
scores = []
for l in jsonlFile.splitlines():
    j = json.loads(l)
    e = j['ESGScores']
    scores.append((j['StatementDetails']['OrganizationId'],
        j['StatementDetails']['FinancialPeriodFiscalYear'],
        e['ESGCombinedScore']['Value'], 
        e['ESGScore']['Value'],
        e['EnvironmentPillarScore']['Value'],
        e['ESGResourceUseScore']['Value'],
        e['ESGEmissionsScore']['Value'],
        e['ESGInnovationScore']['Value'],
        e['SocialPillarScore']['Value'],
        e['ESGWorkforceScore']['Value'],
        e['ESGHumanRightsScore']['Value'],
        e['ESGCommunityScore']['Value'],
        e['ESGProductResponsibilityScore']['Value'],
        e['GovernancePillarScore']['Value'],
        e['ESGManagementScore']['Value'],
        e['ESGShareholdersScore']['Value'],
        e['ESGCsrStrategyScore']['Value'],
        e['ESGCControversiesScore']['Value']))


print('Loaded {} scores'.format(len(scores))) 


In [ ]:
# load the dataset into a pandas dataframe
df2 = pd.DataFrame(scores, columns=['OrganizationId', 'FiscalYear', 'ESGCombinedScore', 'ESGScore', 'EnvironmentPillarScore', 'ESGResourceUseScore', 'ESGEmissionsScore', 'ESGInnovationScore', 'SocialPillarScore', 'ESGWorkforceScore', 'ESGHumanRightsScore', 'ESGCommunityScore', 'ESGProductResponsibilityScore', 'GovernancePillarScore', 'ESGManagementScore', 'ESGShareholdersScore', 'ESGCsrStrategyScore', 'ESGCControversiesScore'])
# change the Fiscal Year data type to a number
df2['FiscalYear'] = df2['FiscalYear'].astype(int)
# keep the latest ESG scores only
df2 = df2.loc[df2.groupby(['OrganizationId'])['FiscalYear'].idxmax()].reset_index(drop=True)
# save the database
df2.to_pickle('ESGScores.pkl')

### Load the pre-downloaded database for Symbology mapping and ESG

In [ ]:
bMapping = pd.read_pickle('Bond_Parent_mapping.pkl')
bMapping.head()

In [ ]:
scores = pd.read_pickle('ESGScores.pkl').astype({'ESGCombinedScore': float, 'ESGScore': float, 'EnvironmentPillarScore': float, 'ESGResourceUseScore': float, 'ESGEmissionsScore': float, 'ESGInnovationScore': float, 'SocialPillarScore': float, 'ESGWorkforceScore': float, 'ESGHumanRightsScore': float, 'ESGCommunityScore': float, 'ESGProductResponsibilityScore': float, 'GovernancePillarScore': float, 'ESGManagementScore': float, 'ESGShareholdersScore': float, 'ESGCsrStrategyScore': float, 'ESGCControversiesScore': float})
scores.head()

### Get the Bond portfolio holdings

In [ ]:
# what is the Lipper ID of the bonds portfolio
portfolioID = 60000170

In [ ]:
# get the constituents bonds in this portfolio
hResp = getRequest('/data/funds/v1/assets/' + str(portfolioID), {'properties': 'holdings'})
print(hResp)

In [ ]:
allHoldings = []
# extract the ISIN, and weights of the bond holdings
for a in hResp['assets'][0]['holdings'][0]['constituents']:
    if 'crossReferenceCodes' in a:
        for code in a['crossReferenceCodes']:
            if code['code'] == 'ISIN':
                allHoldings.append((code['values'][0]['value'], a['weight']))

display(allHoldings[:10])
print('This fund contains {} bonds'.format(len(allHoldings)))

### Match the ESG-Parent company of these bonds

In [ ]:
# create a master dataframe for all processing
mdf = pd.DataFrame(allHoldings, columns =['Bond', 'Weight'])
# merge the ESG parent company info into this dataframe
mdf = mdf.merge(bMapping, how='left', left_on='Bond', right_on='Bond')
display(mdf)

In [ ]:
total = len(allHoldings)
covered = len(mdf['ESGParent'].dropna())
coverage = (covered / total) * 100
fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = coverage,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': 'Coverage % ({} out of {} have ESG data)'.format(covered, total) },
    gauge = {'axis': {'range': [None, 100]}}))

fig.show()

### Calculate and display the consolidated ESG Score for the whole portfolio

In [ ]:
# formulate everything onto a dataframe and display
combined = mdf.merge(scores, how='left', left_on='ESGParent', right_on='OrganizationId')
combined.drop('OrganizationId', axis=1, inplace=True)
display(combined)

In [ ]:
# Rebase, calculate the combined ESG scores of these holdings
weightedSeries = []
for idx, a in combined['ESGCombinedScore'].items():
    if math.isnan(a):
        weightedSeries.append(0)
    else:
        weightedSeries.append(combined['Weight'][idx])

weightTotal = sum(weightedSeries)
rebasedWeight = combined['Weight']/weightTotal

In [ ]:
# calculate the weighted total for the holdings
total = []
for col in combined:
    if col == 'Bond':
        total.append('WEIGHTED AVERAGE')
    elif col == 'Weight':
        total.append(1.0)
    elif col == 'FiscalYear':
        total.append('')
    elif combined[col].dtype == 'float64':
        total.append((combined[col] * rebasedWeight).sum())
    else:
        total.append('')

In [ ]:
# insert the final result into the portfolio
combined.loc[-1] = total
combined.index = combined.index + 1
combined = combined.sort_index()

In [ ]:
# display the final dataframe
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option("display.precision", 2)
display(combined.fillna(''))

In [ ]:
combined.to_clipboard()